# core

> All the core components are stored here.

In [ ]:
#| default_exp core

In [ ]:
#| hide
! rm -rf another-demo/ another-demo.sqlite demo/ demo.sqlite

In [ ]:
#| hide
#| export
from nbdev.showdoc import *
from fastcore.basics import patch
import json
from diskcache import Cache
import hashlib
import orjson
import sqlite_vec
import pysqlite3
from typing import Dict

from typing import List
import struct


def serialize_f32(vector: List[float]) -> bytes:
    """Serializes a list of floats into a compact "raw bytes" format."""
    return struct.pack("%sf" % len(vector), *vector)

In [ ]:
#| export

class VectoyLite:
    def __init__(self, path: str):
        """
        Initializes the VectoLite instance with a connection to the SQLite database.

        Parameters
        ----------
            path 
                The path to the SQLite database file.
        """
        self.path = path
        self.db = pysqlite3.connect(f'{path}.sqlite')
        self.db.enable_load_extension(True)
        sqlite_vec.load(self.db)
        self.db.enable_load_extension(False)
        self.cache = Cache(path)
        self.rownums = None
        self.table_name = 'myvecs'

    def __len__(self):
        """
        Returns the number of rows in the specified table. Will also cache the number of rows internally when called.

        Returns
        -------
        int
            The number of rows in the table.
        """
        if not self.rownums:
            self.rownums = self.db.execute(f"SELECT COUNT(*) FROM {self.table_name}").fetchone()[0]
        return self.rownums

    @property
    def table_exists(self) -> bool:
        """
        Checks if a table exists in the SQLite database.

        Returns
        -------
        bool
            True if the table exists, False otherwise.
        """
        return self.db.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{self.table_name}'").fetchone() is not None

In [ ]:
#| export

@patch
def print_version(self: VectoyLite):
    """
    Prints the SQLite and SQLite-vec versions.
    """
    sqlite_version, vec_version = self.db.execute(
        "select sqlite_version(), vec_version()"
    ).fetchone()
    print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")

NameError: name 'patch' is not defined

In [ ]:
VectoyLite("demo").print_version()

sqlite_version=3.46.1, vec_version=v0.1.1


In [ ]:
#| export

@patch
def parse_item(self: VectoyLite, item: Dict):
    """
    Parses an item and returns its MD5 hash, serialized contents, and vector.

    This is mainly meant as an internal method, but there may be times when you want to confirm these manually.

    Parameters
    ----------
        item 
            The item to parse.

    Returns
    -------
        tuple
            A tuple containing the MD5 hash (str), serialized contents (str), and serialized vector (bytes).
    """
    contents = orjson.dumps({k: v for k, v in item.items() if k != 'vector'})
    md5_hash = hashlib.md5(contents).hexdigest()
    return md5_hash, contents, item['vector']

In [ ]:
VectoyLite("demo").parse_item({"hello": "world", "vector": [1, 2, 3]})

('fbc24bcc7a1794758fc1327fcfebdaf6', b'{"hello":"world"}', [1, 2, 3])

In [ ]:
VectoyLite("demo").parse_item({"hello": "world!", "vector": [1, 2, 3]})

('a8af219242bb71a1c085ee0c7e16e322', b'{"hello":"world!"}', [1, 2, 3])

In [ ]:
#| export

@patch
def insert(self: VectoyLite, stream):
    """
    Inserts a stream of items into the specified table.

    Parameters
    ----------
        stream
            An iterable stream of dictionaries to insert.
    """
    with self.db:
        for item in stream:
            md5_hash, contents, vector = self.parse_item(item)
            print(md5_hash, contents, vector)
    
            # Edge case: if the table does not exist, create it
            if not self.table_exists:
                self.db.execute(f"CREATE VIRTUAL TABLE {self.table_name} USING vec0(embedding float[{len(vector)}])")
                self.rownums = 0

            # If we have already inserted this item, no need to add again
            if md5_hash in self.cache:
                return

            # Insert the item into the table
            i = len(self) + 1
            self.db.execute(
                f"INSERT INTO {self.table_name}(rowid, embedding) VALUES (?, ?)",
                [i, serialize_f32(vector)],
            )
            self.cache[i] = contents
            self.cache[md5_hash] = i
            self.rownums += 1

In [ ]:
VectoyLite("demo").insert([{"hello": "world!", "vector": [1, 2, 3, 4]}])

a8af219242bb71a1c085ee0c7e16e322 b'{"hello":"world!"}' [1, 2, 3, 4]


In [ ]:
#| export

@patch
def query_idx(self: VectoyLite, query, k=5):
    """
    Queries the specified table for the nearest neighbors to the given query vector.

    Parameters
    ----------
        query
            The query vector

    Returns
    -------
        tuple: A tuple containing the rowids and distances of the nearest neighbors.
    """
    results = self.db.execute(
        f"""
          SELECT
            rowid,
            distance
          FROM {self.table_name}
          WHERE embedding MATCH ?
          ORDER BY distance
          LIMIT {k}
        """,
        [serialize_f32(query)],
    ).fetchall()
    return list(zip(*results))

In [ ]:
#| export

@patch
def query(self: VectoyLite, query, k=5):
    """
    Queries the specified table for the nearest neighbors to the given query vector.

    Parameters
    ----------
        query
            The query vector

    Returns
    -------
        tuple: A tuple containing the inserted items and distances of the nearest neighbors.
    """
    idxs, dists = self.query_idx(query, k)
    return [json.loads(self.cache[i].decode()) for i in idxs], dists

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| hide
! rm -rf another-demo/ another-demo.sqlite demo/ demo.sqlite